In [63]:
import sys
from pathlib import Path
import tempfile

import pysal
import pandas as pd
import numpy as np

DIR = Path('..')
sys.path.append(str(DIR))

import arcgdfconvertor as agc


DATA_DIR = DIR/'data/'
OUT_DIR = DIR/'output/'

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
cities = [
    'Auckland',
    'Brisbane',
    'Perth',
    'Vancouver',
]

In [87]:
hex_low_percentage_dict = dict()
for city in cities:

    #Read CSVs Dist_decay
    access_df = pd.read_csv(
        DATA_DIR/'OD_Summaries'/"{0}_Hex_dist_decay_summary.csv".format(city),
        index_col=0, dtype = {'from': str})
    census_gdf = agc.fc_to_gdf(
        str(DATA_DIR/'Cities.gdb'/'{0}_Hex_Polygon'.format(city)))

    cond = (census_gdf['Median_Income'] > 3) & (census_gdf['POP'] > 3) & (census_gdf['POW'] > 3)
    census_gdf = census_gdf[cond].copy()

    access_gdf = census_gdf.merge(
        access_df, 
        left_on = 'Unique_ID', 
        right_on = 'from', 
        how = 'inner')
    hex_total = access_gdf.shape[0]

    # Calculate the weight
    tmp_dir = tempfile.TemporaryDirectory() #make a temporary directory
    p = Path(tmp_dir.name)
    n = city + '.shp'
    access_gdf[['Unique_ID', 'geometry']].to_file(str(p/n))  #make a temporary shapefile
    w = ps.weights.Queen.from_shapefile(str(p/n))
    tmp_dir.cleanup()  # Delete temporary directory

    #Calculate X
    x = np.array(access_gdf['Median_Income'])
    
    #Calculate Y
    hex_low_percentage_list = list()
    for i in range(20, 65, 5):
        y = np.array(access_gdf['dist_decay_{0}'.format(i)])
        lm = ps.esda.moran.Moran_Local_BV(x, y, w, transformation = "r",permutations = 99)
        access_gdf['LISA_CL_{0}'.format(i)] = lm.q
        access_gdf['LISA_P_{0}'.format(i)] = lm.p_sim
        
        cond = ((access_gdf['LISA_P_{0}'.format(i)] <0.05) & 
                (access_gdf['LISA_CL_{0}'.format(i)]==3))
        
        hex_low = access_gdf[cond].shape[0]
        hex_low_percentage = hex_low/hex_total*100
        hex_low_percentage_list.append(hex_low_percentage)
    
    hex_low_percentage_dict[city] = hex_low_percentage_list
    shp_name = city+'.shp'
    #access_gdf.to_file(str(DATA_DIR/'LISA'/shp_name))

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\weights\weights.py:189: UserWarning: There are 4 disconnected observations
  warnings.warn("There are %d disconnected observations" % ni)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\weights\weights.py:190: UserWarning: Island ids: 4, 132, 518, 543
  warnings.warn("Island ids: %s" % ', '.join(str(island) for island in self.islands))
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\esda\moran.py:1089: RuntimeWarning: invalid value encountered in true_divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: Runtime

('WARNING: ', 4, ' is an island (no neighbors)')
('WARNING: ', 132, ' is an island (no neighbors)')
('WARNING: ', 518, ' is an island (no neighbors)')
('WARNING: ', 543, ' is an island (no neighbors)')


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\weights\weights.py:186: UserWarning: There is one disconnected observation (no neighbors)
  warnings.warn("There is one disconnected observation (no neighbors)")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\weights\weights.py:187: UserWarning: Island id: 121
  warnings.warn("Island id: %s" % str(self.islands[0]))


('WARNING: ', 121, ' is an island (no neighbors)')


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\weights\weights.py:189: UserWarning: There are 2 disconnected observations
  warnings.warn("There are %d disconnected observations" % ni)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pysal\weights\weights.py:190: UserWarning: Island ids: 15, 128
  warnings.warn("Island ids: %s" % ', '.join(str(island) for island in self.islands))


('WARNING: ', 15, ' is an island (no neighbors)')
('WARNING: ', 128, ' is an island (no neighbors)')


In [88]:
pd.DataFrame(hex_low_percentage_dict)

,Auckland,Brisbane,Perth,Vancouver
0,7.255521,7.715582,11.192661,1.483051
1,7.255521,7.564297,12.293578,2.118644
2,7.255521,8.623298,12.110092,1.906780
3,9.463722,9.984871,12.660550,2.754237
4,8.517350,11.043873,12.110092,3.601695
5,9.463722,14.523449,13.211009,4.237288
6,8.201893,15.885023,12.293578,4.237288
7,8.517350,16.490166,12.293578,4.661017
8,7.255521,16.792738,12.844037,4.449153
